In [1]:
##Robs summariser of RC9 Additional Scenario Regional Contributor tables
import pandas as pd
import numpy as np
import libarchive
import tempfile
import os
from functools import reduce

#mainPath = 'P:\projects\RC9_ResultsSets\Tester'
mainPath = r'\\athenasmb\pdreef\RC9_RC2017\AdditionalScenarios\Results\ReportCardSpan_1986_2014'
#mainPath = r'\\athenasmb\pdreef\RC9_RC2017\RC9_ResultsSets_PointOfTruth'
#outPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
outPath = 'P:\projects\RC9_ResultsSets\AdditionalScenarioSummaries'
summariesOutFolder = 'SummariesWithProcess'
baseFileName = 'ScenarioBaseline'
D2CFileName = 'Scenario_DtoC'
AllBFileName = 'Scenario_AllB'
AllAFileName = 'Scenario_AllA'

regContributorFileName = 'RegContributorDataGrid.csv'
fuAreasFileName = 'fuAreasTable.csv'
#outputCSVEnd = 'RegToOutlet_Alt.csv'
outputCSVEnd = 'RegToOutlet.csv'
baselineScenarioName = 'Baseline'
changeScenarioName = 'Change'
predevScenarioName = 'PreDev'
reportCardString = 'RC2017'

preDevCSVLoc = 'P:\projects\RC9_ResultsSets\RegContribTables\Summaries'

theTempDir = tempfile.gettempdir()
theCurrentDir = os.getcwd()

print("Current dir is: " + theCurrentDir)

pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC9_RC2017\RC9_ResultsSets_PointOfTruth\constituentsToGroups.csv'

annLoadToStream = 'AnnLoadToStreamKG'
annLoadToExport = 'AnnLoadToExportKG'

scenarioList = [baseFileName, D2CFileName, AllBFileName, AllAFileName]
#regList = ['BM','CY','FI','MW','WT']
regExtDict = {
                'BM':'7z',
               'BU':'zip',
               'CY':'7z',
               'FI':'7z',
               'MW':'7z',
               'WT':'7z'
              }

#scenarioToFolderDict = {baselineScenarioName:baseFolderEnd, changeScenarioName:changeFolderEnd, predevScenarioName:predevFolderEnd}


Current dir is: C:\PythonScriptingForGBR\reef_scripts\ResultsAnalysis


In [2]:
def getPathInfo(regionIDString, scenarioString):
    #fileIn = mainPath + '\\' + regionIDString + '\\Model_Outputs\\' + regionIDString + '_' + scenarioToFolderDict[scenarioString] + '\\' + regContributorFileName
    #fileOut = outPath + '\\' + summariesOutFolder + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    fileIn = mainPath + '\\' + regionIDString + '_RC9_' + scenarioString + '_1986_2014.' + regExtDict[regionIDString]
    fileOut = outPath + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut


In [75]:
#Build relationships of constituent names to groups
#tssGrp = 'TSS'
#tnGrp = 'TN'
#tpGrp = 'TP'
#ps2Grp = 'PSII'
#nonps2Grp = 'non-PSII'
##Nope, will read in a CSV to DataFrame instead, thus joing will 'drop' flow from this comparison
#constToGrp = {'Sediment - Fine':tssGrp}

constToGrp = pd.read_csv(pathToContstitsToGrpFile)

baseSupplyStr = 'Base_Supply_KG'
baseExportStr = 'Base_Export_KG'
regbaseExportStr = 'Reg_Base_Export_KG'
D2CSupplyStr = 'D2C_Supply_KG'
D2CExportStr = 'D2C_Export_KG'
regD2CExportStr = 'Reg_D2C_Export_KG'
D2CPercRedStr = 'D2C_PercRed'
AllBSupplyStr = 'AllB_Supply_KG'
AllBExportStr = 'AllB_Export_KG'
regAllBExportStr = 'Reg_AllB_Export_KG'
AllBPercRedStr = 'AllB_PercRed'
AllASupplyStr = 'AllA_Supply_KG'
AllAExportStr = 'AllA_Export_KG'
regAllAExportStr = 'Reg_AllA_Export_KG'
AllAPercRedStr = 'AllA_PercRed'
predevSupplyStr = 'PreDev_Supply_KG'
predevExportStr = 'PreDev_Export_KG'
regpredevExportStr = 'Reg_PreDev_Export_KG'

D2CRedOnly = 'D2CREDONLYAMT'
AllBRedOnly = 'AllBREDONLYAMT'
AllARedOnly = 'AllAREDONLYAMT'

D2CRedOnlyTotal = 'D2CREDONLYTOT'
AllBRedOnlyTotal = 'AllBREDONLYTOT'
AllARedOnlyTotal = 'AllAREDONLYTOT'

D2CRedAttributed = 'D2CREDONLYATT'
AllBRedAttributed = 'AllBREDONLYATT'
AllARedAttributed = 'AllAREDONLYATT'


repRegStr = 'Rep_Region'
sourceRegStr = 'Source Region'
constituentStr = 'Constituent'
fuStr = 'FU'
groupStr = 'GROUP'
modelRegStr = 'MODELREG'
#percRedStr = 'PercReduction'
regTotalStr = 'REGIONAL TOTAL'
processStr = 'Process'

#allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr])
#allExportsAndReductions = pd.DataFrame()


In [76]:
def getPathInfoForReductions(regionIDString, scenarioString):
    fileIn = mainPath + '\\' + regionIDString + '_RC9_' + scenarioString + '_1986_2014.7z'
    fileOut = outPath + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut

In [77]:
def getPreDevPathInfo(regionIDString, scenarioString):
    fileIn = mainPath + '\\' + regionIDString + '_RC9_' + scenarioString + '_1986_2014.7z'
    fileOut = preDevCSVLoc + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut

In [86]:
#So now we would be reading in known summary files

def calculateReductions(regName, thisRegID):
    
        
    baseOutDir = outPath + '\\' + summariesOutFolder
    if not os.path.exists(baseOutDir):
        os.makedirs(baseOutDir)
        print("Made dir: " + baseOutDir)
    
    
    unneededFile, theBaseSummaryFile = getPathInfoForReductions(thisRegID, baseFileName)
    unneededFile, theD2CSummaryFile = getPathInfoForReductions(thisRegID, D2CFileName)
    unneededFile, theAllBSummaryFile = getPathInfoForReductions(thisRegID, AllBFileName)
    unneededFile, theAllASummaryFile = getPathInfoForReductions(thisRegID, AllAFileName)
    unneededFile, thePreDevSummaryFile = getPreDevPathInfo(thisRegID, predevScenarioName)
    
    #print("Reading: " + theBaseSummaryFile)
    baseSummaryDF = pd.read_csv(theBaseSummaryFile)
    #print("Reading: " + theChangeSummaryFile)
    D2CSummaryDF = pd.read_csv(theD2CSummaryFile)
    AllBSummaryDF = pd.read_csv(theAllBSummaryFile)
    AllASummaryDF = pd.read_csv(theAllASummaryFile)
    #print("Reading: " + thePreDevSummaryFile)
    predevSummaryDF = pd.read_csv(thePreDevSummaryFile)
    
    #inner drops out those that don't merge, left keep them
    baseTotals = pd.merge(baseSummaryDF.groupby([repRegStr,constituentStr,fuStr, processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum', 'AreaHA':'first'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    baseTotals = baseTotals.rename(columns={repRegStr:sourceRegStr,'AreaHA':'AREA_HA',annLoadToStream:baseSupplyStr, annLoadToExport:baseExportStr})
    baseTotals = baseTotals[[sourceRegStr, groupStr, constituentStr, fuStr, processStr, 'AREA_HA', baseSupplyStr, baseExportStr]]
    #baseTotals.head(40)
    
    predevTotals = pd.merge(predevSummaryDF.groupby([repRegStr,constituentStr,fuStr, processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    predevTotals = predevTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:predevSupplyStr, annLoadToExport:predevExportStr})
    predevTotals = predevTotals[[sourceRegStr, constituentStr, fuStr, processStr, predevSupplyStr, predevExportStr]]
    
    D2CTotals = pd.merge(D2CSummaryDF.groupby([repRegStr,constituentStr,fuStr, processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    D2CTotals = D2CTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:D2CSupplyStr, annLoadToExport:D2CExportStr})
    D2CTotals = D2CTotals[[sourceRegStr, constituentStr, fuStr, processStr, D2CSupplyStr, D2CExportStr]]
    
    AllBTotals = pd.merge(AllBSummaryDF.groupby([repRegStr,constituentStr,fuStr, processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    AllBTotals = AllBTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:AllBSupplyStr, annLoadToExport:AllBExportStr})
    AllBTotals = AllBTotals[[sourceRegStr, constituentStr, fuStr, processStr, AllBSupplyStr, AllBExportStr]]
    
    AllATotals = pd.merge(AllASummaryDF.groupby([repRegStr,constituentStr,fuStr, processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    AllATotals = AllATotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:AllASupplyStr, annLoadToExport:AllAExportStr})
    AllATotals = AllATotals[[sourceRegStr, constituentStr, fuStr, processStr, AllASupplyStr, AllAExportStr]]
    
    #changeTotals.head(8)
    
    reductTotalsDF = [D2CTotals, AllBTotals, AllATotals]
    reductionsMerged = reduce(lambda  left,right: pd.merge(left,right,on=[sourceRegStr, constituentStr, fuStr, processStr],
                                                           how='outer'), reductTotalsDF)
    
    #, columns=['Rep_Region', 'GROUP', 'Constituent', 'FU', 'AREA_HA', 'Base_Supply_KG', 'Base_Export_KG', 'PreDev_Supply_KG', 'PreDev_Export_KG', 'Change_Supply_KG', 'Change_Export_KG']
    anthroRegByLuseProcess = pd.merge(pd.merge(baseTotals, predevTotals, how='left', left_on=[sourceRegStr,constituentStr,fuStr, processStr],
                                               right_on = [sourceRegStr,constituentStr,fuStr, processStr]).reset_index(), reductionsMerged,
                                      how='left', left_on=[sourceRegStr,constituentStr,fuStr, processStr], right_on = [sourceRegStr,constituentStr,fuStr, processStr]).reset_index()
    #anthroRegByLuse.head(8)
    
    ##Need to add in a few export differences for totalising
    anthroRegByLuseProcess[D2CRedOnly] = np.where(anthroRegByLuseProcess[D2CExportStr] < anthroRegByLuseProcess[baseExportStr],
                                                  anthroRegByLuseProcess[baseExportStr] - anthroRegByLuseProcess[D2CExportStr],0)
    anthroRegByLuseProcess[AllBRedOnly] = np.where(anthroRegByLuseProcess[AllBExportStr] < anthroRegByLuseProcess[baseExportStr],
                                                   anthroRegByLuseProcess[baseExportStr] - anthroRegByLuseProcess[AllBExportStr],0)
    anthroRegByLuseProcess[AllARedOnly] = np.where(anthroRegByLuseProcess[AllAExportStr] < anthroRegByLuseProcess[baseExportStr],
                                                   anthroRegByLuseProcess[baseExportStr] - anthroRegByLuseProcess[AllAExportStr],0)
    
    #Lets add the totals and prec reduction estimates before exporting
    #theFileOut = baseOutDir + '\\' + thisRegID + '_RC2017_AddScenarios_AnthroRegByLanduseProcess.csv'
    #anthroRegByLuseProcess.to_csv(theFileOut, index=False)
    ##print("Saved " + theFileOut)
    
    regExports = anthroRegByLuseProcess.groupby([sourceRegStr, constituentStr, groupStr]).agg({predevExportStr:'sum',
                                                                                               baseExportStr:'sum',
                                                                                               D2CExportStr:'sum',
                                                                                               AllBExportStr:'sum',
                                                                                               AllAExportStr:'sum',
                                                                                               D2CRedOnly:'sum',
                                                                                               AllBRedOnly:'sum',
                                                                                               AllARedOnly:'sum'})
    
    regExports[modelRegStr] = regName
    regExports[D2CPercRedStr] = (regExports[baseExportStr] - regExports[D2CExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    regExports[AllBPercRedStr] = (regExports[baseExportStr] - regExports[AllBExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    regExports[AllAPercRedStr] = (regExports[baseExportStr] - regExports[AllAExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    
    regExports = regExports.rename(columns={predevExportStr:regpredevExportStr,baseExportStr:regbaseExportStr,
                                            D2CExportStr:regD2CExportStr, AllBExportStr:regAllBExportStr,
                                            AllAExportStr:regAllAExportStr, D2CRedOnly:D2CRedOnlyTotal,
                                           AllBRedOnly:AllBRedOnlyTotal, AllARedOnly:AllARedOnlyTotal})
    
    regExports = regExports.reset_index()
    regExports = regExports[[modelRegStr, sourceRegStr, constituentStr, groupStr, regpredevExportStr, regbaseExportStr,
                             regD2CExportStr, D2CPercRedStr, regAllBExportStr, AllBPercRedStr, regAllAExportStr, AllAPercRedStr,
                            D2CRedOnlyTotal, AllBRedOnlyTotal, AllARedOnlyTotal]]
    #print(regExports.head(15))
    
    #keepp groupStr in join, so as not to duplicate columns
    anthroRegByLuseProcess = pd.merge(anthroRegByLuseProcess, regExports, how='left',
                                      left_on=[sourceRegStr,groupStr,constituentStr], right_on = [sourceRegStr,groupStr,constituentStr])
    
    
    #Now some final assessment of how much reduction is attributable to each FU/process combination
    #D2CRedAttributed
    
    anthroRegByLuseProcess[D2CRedAttributed] = np.where((anthroRegByLuseProcess[D2CPercRedStr] > 0) & (anthroRegByLuseProcess[D2CRedOnlyTotal] > 0),
                                                       (anthroRegByLuseProcess[D2CRedOnly] / anthroRegByLuseProcess[D2CRedOnlyTotal]) * anthroRegByLuseProcess[D2CPercRedStr],
                                                       0)
    
    anthroRegByLuseProcess[AllBRedAttributed] = np.where((anthroRegByLuseProcess[AllBPercRedStr] > 0) & (anthroRegByLuseProcess[AllBRedOnlyTotal] > 0),
                                                       (anthroRegByLuseProcess[AllBRedOnly] / anthroRegByLuseProcess[AllBRedOnlyTotal]) * anthroRegByLuseProcess[AllBPercRedStr],
                                                       0)
    anthroRegByLuseProcess[AllARedAttributed] = np.where((anthroRegByLuseProcess[AllAPercRedStr] > 0) & (anthroRegByLuseProcess[AllARedOnlyTotal] > 0),
                                                       (anthroRegByLuseProcess[AllARedOnly] / anthroRegByLuseProcess[AllARedOnlyTotal]) * anthroRegByLuseProcess[AllAPercRedStr],
                                                       0)

    #print(anthroRegByLuseProcess.head(15))
    #return

    #thisIndex = [sourceRegStr, groupStr, constituentStr, fuStr, processStr,
    #                                                 'AREA_HA', baseSupplyStr, baseExportStr, predevSupplyStr, predevExportStr,
    #                                                D2CSupplyStr, D2CExportStr, AllBSupplyStr, AllBExportStr, AllASupplyStr, AllAExportStr,
    #                                                D2CPercRedStr, AllBPercRedStr, AllAPercRedStr]
    #anthroRegByLuseProcess.index = thisIndex
    
    anthroRegByLuseProcess = anthroRegByLuseProcess[[sourceRegStr, groupStr, constituentStr, fuStr, processStr,
                                                     'AREA_HA', baseSupplyStr, baseExportStr, predevSupplyStr, predevExportStr,
                                                    D2CSupplyStr, D2CExportStr, AllBSupplyStr, AllBExportStr, AllASupplyStr, AllAExportStr,
                                                    regpredevExportStr, regbaseExportStr, regD2CExportStr, regAllBExportStr, regAllAExportStr,
                                                    D2CPercRedStr, AllBPercRedStr, AllAPercRedStr,
                                                    D2CRedOnly, AllBRedOnly, AllARedOnly,
                                                    D2CRedOnlyTotal, AllBRedOnlyTotal, AllARedOnlyTotal,
                                                    D2CRedAttributed, AllBRedAttributed, AllARedAttributed]]
    
        
    theFileOut = baseOutDir + '\\' + thisRegID + '_RC2017_AddScenarios_AnthroRegByLanduseProcess.csv'
    anthroRegByLuseProcess.to_csv(theFileOut, index=False)
    
    modelRegExports = regExports.groupby([modelRegStr, constituentStr, groupStr]).agg({regpredevExportStr:'sum',
                                                                                       regbaseExportStr:'sum',
                                                                                       regD2CExportStr:'sum',
                                                                                       regAllBExportStr:'sum',
                                                                                       regAllAExportStr:'sum',
                                                                                      D2CRedOnlyTotal:'sum',
                                                                                      AllBRedOnlyTotal:'sum',
                                                                                      AllARedOnlyTotal:'sum'})
    modelRegExports[sourceRegStr] = regTotalStr
    modelRegExports[D2CPercRedStr] = (modelRegExports[regbaseExportStr] - modelRegExports[regD2CExportStr]).div(modelRegExports[regbaseExportStr] - modelRegExports[regpredevExportStr]).mul(100)
    modelRegExports[AllBPercRedStr] = (modelRegExports[regbaseExportStr] - modelRegExports[regAllBExportStr]).div(modelRegExports[regbaseExportStr] - modelRegExports[regpredevExportStr]).mul(100)
    modelRegExports[AllAPercRedStr] = (modelRegExports[regbaseExportStr] - modelRegExports[regAllAExportStr]).div(modelRegExports[regbaseExportStr] - modelRegExports[regpredevExportStr]).mul(100)
    modelRegExports = modelRegExports.reset_index()
    modelRegExports = modelRegExports[[modelRegStr, sourceRegStr, constituentStr, groupStr, regpredevExportStr, regbaseExportStr,
                                       regD2CExportStr, D2CPercRedStr, regAllBExportStr, AllBPercRedStr, regAllAExportStr, AllAPercRedStr,
                                      D2CRedOnlyTotal, AllBRedOnlyTotal, AllARedOnlyTotal]]
    ###modelRegExports.head(15)
    regExports = regExports.append(modelRegExports, ignore_index=True)
    
    return regExports


In [87]:
allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr,
                                                baseExportStr, D2CExportStr, D2CPercRedStr, AllBExportStr, AllBPercRedStr,
                                                AllAExportStr, AllAPercRedStr])

#theseExports = calculateReductions('Burdekin', 'BU')
#allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Burnett Mary', 'BM')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Burdekin', 'BU')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Cape York', 'CY')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Fitzroy', 'FI')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Mackay Whitsunday', 'MW')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theseExports = calculateReductions('Wet Tropics', 'WT')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

#theFileNameOut = outPath + '\\' + summariesOutFolder + '\\AllExportsAndReductions.csv'
#allExportsAndReductions.to_csv(theFileNameOut, index=False)
allExportsAndReductions.head(20)
#print(allExportsAndReductions)
print("Finished")

Finished
